In [2]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import matplotlib.pyplot as plt
from tensorflow.keras.applications.vgg16 import preprocess_input
import cv2

In [3]:
# Параметры
IMG_SIZE = 200
MODEL_PATH = 'imagenet_classifier.h5'
DATA_PATH = 'эD:/Soft/kagglecatsanddogs_3367a/lab3/var5'  # Путь к папке с данными

In [4]:
# Шаг 1: Загрузка модели
model = load_model(MODEL_PATH)
print("Model loaded successfully.")

# Функция для визуализации активаций
def visualize_activations(model, image_path, layer_names):
    img = load_img(image_path, target_size=(IMG_SIZE, IMG_SIZE))
    img_array = img_to_array(img)
    img_array = preprocess_input(np.expand_dims(img_array, axis=0))

    layer_outputs = [layer.output for layer in model.layers if layer.name in layer_names]
    activation_model = tf.keras.models.Model(inputs=model.input, outputs=layer_outputs)
    activations = activation_model.predict(img_array)

    for layer_name, activation in zip(layer_names, activations):
        num_filters = activation.shape[-1]
        size = activation.shape[1]
        n_cols = num_filters // 8
        display_grid = np.zeros((size * n_cols, 8 * size))

        for col in range(n_cols):
            for row in range(8):
                channel_image = activation[0, :, :, col * 8 + row]
                channel_image -= channel_image.mean()
                channel_image /= (channel_image.std() + 1e-5)
                channel_image *= 64
                channel_image += 128
                channel_image = np.clip(channel_image, 0, 255).astype('uint8')
                display_grid[col * size: (col + 1) * size, row * size: (row + 1) * size] = channel_image

        scale = 1.0 / size
        plt.figure(figsize=(scale * display_grid.shape[1], scale * display_grid.shape[0]))
        plt.title(layer_name)
        plt.grid(False)
        plt.imshow(display_grid, aspect='auto', cmap='viridis')
        plt.show()

Model loaded successfully.


In [5]:
# Шаг 2: Визуализация активаций для одного изображения из каждого класса
class_folders = os.listdir(DATA_PATH)
layer_names = [layer.name for layer in model.layers if 'conv' in layer.name]

for class_folder in class_folders:
    image_path = os.path.join(DATA_PATH, class_folder, os.listdir(os.path.join(DATA_PATH, class_folder))[0])
    print(f"Visualizing activations for class: {class_folder}")
    visualize_activations(model, image_path, layer_names)



OSError: [WinError 123] Синтаксическая ошибка в имени файла, имени папки или метке тома: 'эD:/Soft/kagglecatsanddogs_3367a/lab3/var5'

In [ ]:
# Шаг 3: Визуализация фильтров
for layer in model.layers:
    if 'conv' in layer.name:
        filters, biases = layer.get_weights()
        n_filters = filters.shape[-1]

        for i in range(min(n_filters, 8)):
            f = filters[:, :, :, i]
            f_min, f_max = f.min(), f.max()
            f = (f - f_min) / (f_max - f_min)
            plt.imshow(f[:, :, 0], cmap='viridis')
            plt.title(f"Filter {i} in layer {layer.name}")
            plt.show()
            plt.savefig('test5.png')



In [ ]:
# Шаг 4: Визуализация тепловых карт активации класса
from tensorflow.keras import backend as K

def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    grad_model = tf.keras.models.Model(
        [model.inputs], [model.get_layer(last_conv_layer_name).output, model.output]
    )

    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(predictions[0])
        class_channel = predictions[:, pred_index]

    grads = tape.gradient(class_channel, conv_outputs)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    conv_outputs = conv_outputs[0]
    heatmap = conv_outputs @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)

    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()

last_conv_layer_name = layer_names[-1]

for class_folder in class_folders:
    image_path = os.path.join(DATA_PATH, class_folder, os.listdir(os.path.join(DATA_PATH, class_folder))[0])
    img = load_img(image_path, target_size=(IMG_SIZE, IMG_SIZE))
    img_array = preprocess_input(np.expand_dims(img_to_array(img), axis=0))

    heatmap = make_gradcam_heatmap(img_array, model, last_conv_layer_name)

    img = cv2.imread(image_path)
    img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
    heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))
    heatmap = np.uint8(255 * heatmap)
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)

    superimposed_img = cv2.addWeighted(img, 0.6, heatmap, 0.4, 0)

    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.title("Original Image")

    plt.subplot(1, 2, 2)
    plt.imshow(cv2.cvtColor(superimposed_img, cv2.COLOR_BGR2RGB))
    plt.title("Class Activation Heatmap")
    plt.show()
    plt.savefig('test6')